In [1]:
import os, sys
import shutil
import pandas as pd
import numpy as np

In [2]:
main = os.getcwd()
print('main: ', main)
os.chdir(main + '\\' + 'tests_ignore')
prj = os.getcwd()
print('prj: ', prj)
# os.chdir(main)
# os.getcwd()

main:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad
prj:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore


In [3]:
for root, dirs, files in os.walk(prj):
    print(root)
    print(dirs)
    print(files)

c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore
[]
['1217861-1.dcm', 'ECG_MOUSE-cache.lib', 'ECG_MOUSE.csv', 'ECG_MOUSE.lib', 'ECG_MOUSE.net', 'main_copy.lib', 'orig_ECG_MOUSE', 'test_copy']


In [4]:
shutil.copy2('ECG_MOUSE.csv', 'test_copy')
for root, dirs, files in os.walk(prj):
    print(files)

['1217861-1.dcm', 'ECG_MOUSE-cache.lib', 'ECG_MOUSE.csv', 'ECG_MOUSE.lib', 'ECG_MOUSE.net', 'main_copy.lib', 'orig_ECG_MOUSE', 'test_copy']


## Extract .csv data from file into DataFrame

In [5]:
mpn = 'Manufacturer Part Number'
mpn_ = 'Manufacturer_Part_Number'
mpno = 'Manufacturer Part No.'
manuf = 'Manufacturer'
manuf_ = 'Manufacturer_Name'
ampn = 'Alt. Manufacturer Part Number'
amanuf = 'Alt. Manufacturer'
ref = 'Reference'
ds = 'Datasheet'

with open('test_copy', 'r') as schemtxt:
    data = schemtxt.read().splitlines()
    headers = data[0].split(', ')       #headers need to be labeled headers
    for line in range(0, len(data)):
        # data[line] = data[line].split('","')
            #replace commas with double => prev. issues @mpns w/commas
        data[line] = data[line].replace('","', '",,"')
        data[line] = data[line].split(',,')
        
    data = data[1:len(data)]
    #re-init data w/o headers

    df = pd.DataFrame(data, index = np.arange(1, len(data)+1), columns = headers)
    df[mpn_] = df[mpn_].str.strip('"')
    df[ref] = df[ref].str.strip('"')
    df[ds] = df[ds].str.strip('"')

    # print(df.head(5))


## Clean Dataframe Content

In [6]:
"""
columns already strings => str.cat works (string concatenation)
https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python 
    """
# df = df.rename(columns = {'Part Number' : 'MPN', 
#                             'Alternative Part Number' : 'Alt. MPN',
#                             'Alternative Manufacturer' : 'Alt. Manufacturer'})
################################## PART NUMBERS ##################################

            #combine + delete part no. columns
# if (len(df.columns.values[5]) == 0):

if mpno or mpn_ in df.columns.values:
    df['Part Number'] = df['Part Number'].str.cat(df[[mpno,
    mpn_]])
    df = df.drop([mpn_, mpno], axis=1)
    df = df.rename(columns = {'Part Number' : 'Manufacturer Part Number'})

        #drops the column - df.drop([x1, x2]) allows multiple vs df.pop() and del

################################## MANUF NAME ##################################
#repeat of part numbers
# if (len(df.columns.values[4]) == 0):

if manuf_ in df.columns.values:
    df[manuf] = df[manuf].str.cat(df[manuf_])
    df = df.drop([manuf_], axis=1)

df = df.drop(['Description'], axis=1)
headers = df.columns.values.tolist()


#### At this point, the escheema file with no extension (the un-extensioned .csv file) that contains the schematic listed information for all of the parts on the schematic has been...
* converted into a dataframe - BINGO
  * move on to library
* had extra fields removed - BINGO 
  * move on to library
* successfully interpretable by KiCad - NOPE
  * STOP PURSUIT

Turns out there was an error in how I was running the command line operation for the scripts to convert a schematic into a .csv file. We noticed that we received a .xml file, but after some further research and the __failure of the third bullet__, I discovered that the file with no extension... Was in fact... the .csv file that we were missing. 

* This has now been resolved by adding the needed .csv after the capital O in the command line script. 

In [7]:
print(headers)
print(type(df))
print(df.shape, df.size)

['Reference', 'Value', 'Footprint', 'Datasheet', 'Manufacturer', 'Manufacturer Part Number']
<class 'pandas.core.frame.DataFrame'>
(104, 6) 624


In [8]:
# lib_path = os.getcwd()

# with open('ECG_MOUSE', 'w') as library:
#     # library.write(df.to_string(index = False, na_rep = '<>', col_space = 0))
#     tonk = df.to_csv(na_rep = '', 
#                         index = False,
#                         sep = '^').replace('\n', '\n"')
#     tonk = tonk.replace('^', '","').replace('","', ',', len(df.columns.values.tolist()))
#     library.write(tonk)
# for root, dirs, files in os.walk(lib_path):
#     # print(root)
#     # print(dirs)
#     print(files)

____
## Creating the Library (From .csv)

In [1]:
import os, sys
import shutil
import pandas as pd
import numpy as np

In [2]:
# os.chdir(main)
# os.getcwd()

In [3]:
main = os.getcwd()
print('main: ', main)
os.chdir(main + '\\' + 'tests_ignore')
prj = os.getcwd()
print('prj: ', prj)

main:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad
prj:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore


In [8]:
with open('ECG_MOUSE-cache.lib', 'r') as raw:
    raw = raw.read().split('ENDDEF\n')
    compdetails = dict()
    for item in raw:
        if 'R-Device' in item:
            # resdraw = item update
            compdetails.update({'res' : item})
            #draw resistor
        if 'C-Device' in item:
            # cdraw = item
            compdetails.update({'c' : item})
            #draw capacitor
        if 'C_Small' in item:
            # csmalldraw = item
            compdetails.update({'csmall' : item})
        if 'CP-Device' in item:
            cpdraw = item
            compdetails.update({'cp' : item})
            #draw unpolarized cap
        if 'CP_Small' in item:
            # cpsmalldraw = item
            compdetails.update({'cpsmall' : item})
            ##draw small unpolarized cap
        if 'CP1-Device' in item:
            cpdraw_us = ite/m
            compdetails.update({'cp1' : item})
            #draw us symbol capacitor
        if 'CP1_Small' in item:
            compdetails.update({'cp1small' : item})
            # cp1smalldraw = item

        item = item + 'ENDDEF\n'
# compdetails.update('C0805fp' : 'Capacitors_SMD:C_0805')
# compdetails.update('R0805fp' : 'Resistors_SMD:R_0805')
        #SAAD DIDN'T GIVE ME FOOTPRINT ACCESS
        
# print(type(drawer['res']))

<class 'str'>


Library content was successfully taken out and put into a dictionary containing all of the parts details. Now each dictionary entry needs to be replicated and do the desired thing...

* Create a passive component (Resistor/Capacitor) with the desired symbol
* Assign it a desired value
* Insert the other requested fields
Then a successful library has been generated for import (if header and footer of file are included)

Cool things to do, that would be cool to accelerate this process
* A list of desired 0805 resistor/capacitor values for sweeping
* A list of desired 0605 resistor/capacitor values for sweeping
* The footprint strings that are necessary
    * The managing of libraries would have to be done as well
* Maybe make a webscraping thing to get these links off of digikey for me?
    * Maybe have like a preferred manufacturer?
    * Manufacturer & MPN + Alts?
* Have a setup that can let me sweep through Saad's other projects as well?
    * Full python script that can run in terminal?
    * Institute classes and other stuff
    * Ask for string input for document sweeping and generation?
    * Learn how to implement a GUI?

_______
## Extracting from the Netlist

In [54]:
with open('ECG_MOUSE.net', 'r') as netlist:
    temp = netlist.read()
    startcomp, endcomp = temp.find('(component'), (temp.find('(libparts'))
    # print(type(startcomp), startcomp, endcomp)
    # print(temp[endcomp])
    temp = temp[startcomp:endcomp]
            # the components section has been extracted
    with open('netcomp', 'w') as nc:
        nc.write(temp)
        


<class 'int'> 2010 38231


In [38]:
# ha = 'abcdefghijklmnop\n' + \
        # 'qrstuvwxyz\n' + 'button! up'
# print(ha.find('j', end=ha.find('button!')))
# ha.find('button!')
thing = 'thing'
print(type(thing[0]))

<class 'str'>
